In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [2]:
# os.chdir('../')

In [3]:
import data_util.config
importlib.reload(data_util.config)
from data_util.config import *

In [4]:
!pwd

/Users/rowancassius/Desktop/capstone/LSTM_Summarizer


In [5]:
log_root

'/Users/rowancassius/Desktop/capstone/LSTM_Summarizer'

In [6]:
result_path = os.path.join(log_root, results_path)

In [7]:
result_path

'/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data/results'

In [8]:
df1 = pd.read_csv(os.path.join(result_path, 'results_wpt_heavyupsample.tsv'), sep = '\t')
df2 = pd.read_csv(os.path.join(result_path, 'results_wpt_heavyupsample_please.tsv'), sep = '\t')
df3 = pd.read_csv(os.path.join(result_path, 'results_wptextra_upsample.tsv'), sep = '\t')
df5 = pd.read_csv(os.path.join(result_path, 'test_results_with_art.csv'), sep = '\t')

In [23]:
# use the QA model as a baseline
from allennlp.predictors.predictor import Predictor
import allennlp_models.rc

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo-model-2020.03.19.tar.gz")

TypeError: __init__() got an unexpected keyword argument 'max_n'

In [ ]:
predictor.predict(
  passage="The Matrix is a 1999 science fiction action film written and directed by The Wachowskis, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano.",
  question="Who stars in The Matrix?"
)["best_span_str"]

In [17]:
question = "What should you do?"
def qa_summarize(text):
    return predictor.predict(passage=text, question=question)['best_span_str']

In [55]:
df4 = pd.DataFrame.copy(df3)
df4['decoded'] = [qa_summarize(t) for t in df4['task']]

KeyboardInterrupt: 

In [9]:
pd.set_option('display.max_colwidth', -1)

In [21]:
df4.sample(10)

,context,task,ref,decoded,refWC,decodedWC,taskWC,contextWC
350,"forward a set of minimum requirements hardware.. , softare has been written by me, with some public domain software. I am thinking of setting up an internal website with model output data, and verification info. Cheers, Christian. Christian, Just finished meeting with Pual, Vince & Mark. New Plan:. Let's plan on your coming to Houston March 12th - April 2nd after our summer/winters respectively. BUT.. let's proceed with the project without pause:",1. please send up the software that needs to be installed along with operating system requirements,send software and operating systems requirements to SENDER,send up the software,8,5,16,72
86,"Angela, CO 0413 Cost Center 105657. Taffy,",Can you help Angela with this.,help angela,help Angela,2,2,6,7
304,"Your feedback plays an important role in the performance management process, and your participation is very critical to the success of Enron's Performance Management goals. Please provide feedback on the employee s listed below by accessing the Performance Management System PEP and completing an online feedback form as described in the Performance Management Quick Reference Guide . You may begin your feedback input immediately. Please have all feedback forms completed by the date noted below.","If you have any questions regarding PEP or your responsibility in the process, please call the PEP Help Desk at the following numbers:",call pep help desk,PEP Help Desk,4,5,23,75
615,"Our upstream is Sempra, our Downstream is Mirant, it is a llh schedule that was cut by 1 mw HE 4-6, 23-24. Mirant is taking it to real time, and Snohomish who is generating agrees with this.",Please call Mirant for the hours above to make sure everything is being handled.,call mirant,make sure everything is being handled,2,2,14,37
436,Rod: Is this one ready to go?,Please talk to Susan.,contact susan,talk to Susan.,2,3,4,7
83,"Group,","In addition to Virginia's suggestions below, please copy me when responding to Virginia's DMS emails.",copy SENDER on emails to Virginia,responding to Virginia's DMS emails,6,4,15,1
286,NaN,Andrew Kalotay andy@kalotay.com on 09/11/2000 01:27:39 PM Please respond to andy@kalotay.com To: Vincent Kaminski vkamins@enron.com cc: Subject: More hedge effectiveness testing,respond to andy@kalotaycom,More hedge effectiveness testing,3,3,21,0
182,NaN,"Jeff, can you speak briefly with Kevin on this one - intriguing idea if the execution was probable.",speak with Kevin,execution was probable.,3,3,18,0
378,NaN,"Please call Bruce Silvers with Bingham & Dana in Hartford, Conn, and tell him my trip has rescheduled.",call Bruce Silvers,tell him my trip has rescheduled.,3,9,18,0
184,NaN,"At your earliest convenience, please add the new Storage flag to all your Storage deals.",add new storage flag to storage deals,add the new Storage flag to all your Storage deals,7,7,15,0


In [10]:
from rouge import Rouge

In [73]:
Rouge.AVAILABLE_METRICS

{'rouge-l', 'rouge-n', 'rouge-w'}

In [24]:
# Rouge().get_scores(df1['decoded'].map(lambda x: x.lower()), 
#                    df1['ref'].map(lambda x: x.lower()), 
#                    avg = True)

# R = Rouge(metrics=['rouge-l', 'rouge-2', 'rouge-l'])

# R.get_scores(df5['decoded'].map(lambda x: x.lower()), 
#              df5['ref'].map(lambda x: x.lower()))

Rouge().get_scores(df3['decoded'].map(lambda x: x.lower()), 
                   df3['ref'].map(lambda x: x.lower()), 
                   avg=True)

{'rouge-1': {'f': 0.6438843273617484,
  'p': 0.7028495615495612,
  'r': 0.6513676879765121},
 'rouge-2': {'f': 0.4524710123958966,
  'p': 0.5062263070263071,
  'r': 0.46146350697233046},
 'rouge-l': {'f': 0.6407098745829731,
  'p': 0.7015400488400488,
  'r': 0.6433428749899343}}

In [42]:
Rouge().get_scores('respond to Rogelio', 'respond to Rogelio L', avg = True) 

{'rouge-1': {'f': 0.8571428522448981, 'p': 1.0, 'r': 0.75},
 'rouge-2': {'f': 0.7999999952000001, 'p': 1.0, 'r': 0.6666666666666666},
 'rouge-l': {'f': 0.8571428522448981, 'p': 1.0, 'r': 0.75}}

In [43]:
# prev scores
scores

{'rouge-1': {'f': 0.6246012619259179,
  'p': 0.7307787698412702,
  'r': 0.5949822874954176},
 'rouge-2': {'f': 0.3966156426120457,
  'p': 0.4730038265306122,
  'r': 0.381483452063809},
 'rouge-l': {'f': 0.6180453764345568,
  'p': 0.7225885770975058,
  'r': 0.586240334566227}}

In [56]:
df4.sample(10)

,context,task,ref,decoded,refWC,decodedWC,taskWC,contextWC
146,NaN,can you also add Karen Herrmann to the distribution?,add Karen Herrmann to distribution,add Karen Herrmann to distribution,5,5,9,0
378,NaN,"Please call Bruce Silvers with Bingham & Dana in Hartford, Conn, and tell him my trip has rescheduled.",call Bruce Silvers,call Bruce Silvers with Bingham & Dana in Hartford,3,9,18,0
35,IMPORTANT:,PLEASE READ THE ATTACHED FILE Ethical Wall Procedure AND PRINT AND SIGN THE EMPLOYEE CERTIFICATION below .,"read file , print and sign certification",read attached file,7,3,17,1
172,RE: Notice letter. We sent out a notice letter to GE last Friday regarding closing of a financing.,Could you print me a hard copy so I can show you what changes need to be made?,print copy for SENDER,print sender hard copy,4,4,18,18
440,NaN,please see the dial in number and call info below.,call the dial in number,see dial in number and call info,5,7,10,0
466,"Tana, Kelly, Karen:. The product long description below will fall under the new product type:. 09US Benzene Fin Opt Credit Tom Moran/Wendi Lebrocq have approved copying the profiles for the new product type from the following:. 09US Unl Gasoline Fin Opt","Please respond no later than 10 AM, Wednesday, October 10.",reply to SENDER,respond to sender,3,3,10,41
467,I have added deal # 541123 as a buyback for December 25-31 2000. Please invoice South Hampton total flow thru the meter 12/01 - 12/24; on 12/25 they should be invoiced a minimum of 1700 mmbtu per day at contract and buy back 900 mmbtu per day per sitara 541123.,Please call with questions.,call SENDER,call sender with questions,2,4,4,50
551,"John, I've received internal approval to participate. I'll see what I can come up with for a bio. Thanks, Kay. Kay Mann email ' Kay.Mann@enron.com cc: Keffer, John JKeffer@KSLAW.com. Dear Kay, I was delighted to hear from John Keffer that, subject to receiving internal approvals, you would be joining us for our Roundtable discussion on Thursday.",When you get the necessary approvals could you please send me your bio so that we can make sure you get due credit in The Texas Lawyer?,send bio to SENDER,send bio to sender,4,4,27,56
713,"We estimate that researching some new issues presented by you and preparing a Memorandum of Law, as well as reviewing and revising the ISDA documentation exhibits and delivering opinions will take approximately a week, although we will try to get our reply back to you as soon as possible. We also estimate that our fees would be approximately 3,000 to 5,000, depending on the amount of work actually required to consummate this transaction.",Please do not hesitate to call me should you have any questions.,call SENDER,call sender,2,2,12,73
393,NaN,Please cc the following distribution list with updates:,cc list with updates,cc distribution list,4,3,8,0


### Calculate Individual Scores

In [25]:
def score(r):
    scores = Rouge().get_scores(r.decoded.lower(), r.ref.lower())[0]
    return [scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']]

In [26]:
df1 = pd.concat([df1, pd.DataFrame([score(r) for r in df1.itertuples()], 
                                  columns=['Rouge-1', 'Rouge-2', 'Rouge-L'])], axis=1)

df2 = pd.concat([df2, pd.DataFrame([score(r) for r in df2.itertuples()], 
                                   columns=['Rouge-1', 'Rouge-2', 'Rouge-L'])], axis=1)

df3 = pd.concat([df3, pd.DataFrame([score(r) for r in df3.itertuples()], 
                                   columns=['Rouge-1', 'Rouge-2', 'Rouge-L'])], axis=1)

In [91]:
df1.describe()

,refWC,decodedWC,taskWC,contextWC,Rouge-1,Rouge-2,Rouge-L
count,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,5.186667,4.796000,13.256000,26.169333,0.640206,0.444700,0.638767
std,2.799930,2.250586,7.889664,26.640200,0.278676,0.362389,0.280888
min,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,8.000000,0.000000,0.444444,0.027778,0.422932
50%,5.000000,4.000000,12.000000,16.500000,0.666667,0.400000,0.666667
75%,6.000000,6.000000,17.000000,52.000000,0.888889,0.727273,0.888889
max,28.000000,14.000000,71.000000,75.000000,1.000000,1.000000,1.000000


In [92]:
df2.describe()

,refWC,decodedWC,taskWC,contextWC,Rouge-1,Rouge-2,Rouge-L
count,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,5.186667,4.392000,13.256000,26.169333,0.626773,0.424809,0.627072
std,2.799930,2.434935,7.889664,26.640200,0.285021,0.363771,0.288765
min,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,8.000000,0.000000,0.444444,0.000000,0.400000
50%,5.000000,4.000000,12.000000,16.500000,0.666667,0.369318,0.666667
75%,6.000000,5.000000,17.000000,52.000000,0.857143,0.666667,0.888889
max,28.000000,16.000000,71.000000,75.000000,1.000000,1.000000,1.000000


In [27]:
df3.describe()

,refWC,decodedWC,taskWC,contextWC,Rouge-1,Rouge-2,Rouge-L
count,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,5.186667,4.636000,13.256000,26.169333,0.643884,0.452471,0.640710
std,2.799930,2.254095,7.889664,26.640200,0.281626,0.364565,0.285419
min,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,8.000000,0.000000,0.444444,0.108359,0.428571
50%,5.000000,4.000000,12.000000,16.500000,0.666667,0.400000,0.666667
75%,6.000000,6.000000,17.000000,52.000000,0.888889,0.750000,0.909091
max,28.000000,16.000000,71.000000,75.000000,1.000000,1.000000,1.000000


In [61]:
df = pd.merge(df3, df4, on = ['context', 'task', 'ref'], suffixes=('_3', '_4'))

In [62]:
df[['context', 'task', 'ref', 'decoded_3', 'decoded_4']].sample(10)

,context,task,ref,decoded_3,decoded_4
477,"Hi Dan, I have attached the pending trades our system shows for Reliant-Entex.",Could you review and update me on which confirmations have been sent.,review update SENDER on sent confirmations,review and update sender,review and update sender
618,I have no contacts to buy from. Mountaineer production is normally sold on term deals. CES needs 17dts/day of Mountaineer gas for the rest of March. The request was made by Kim Novesak? ? I will be out this afternoon.,Please call Brian Perrone 703-5616-486 or Scott Goodell at 713-8537-711 .,call perrone Brian Perrone at 703-5616-486 or Scott Goodell at 713-8537-711,call Brian Perrone 703-5616-486 or Scott Goodell at 713-8537-711,call Brian Perrone 703-5616-486 or Scott Goodell at 713-8537-711
425,"Diann,",I am flying back to Portland tomorrow morning at about 9:30 arriving at 12:00 Please call me at 503-8078-959 or email me w/ and update on you talks w/ CSC.,call SENDER at 503-8078-959,call sender at 503-8078-959 or email sender,call sender at 503-8078-959 or email sender
532,That sounds like the LDs on the air filter are accruing.,Please confirm with Kay.,confirm with Kay,confirm with Kay,confirm with Kay
690,"Daren,",Please confirm if you would be able to meet with Bob Hall and Pat Clynes to discuss Producer Services,confirm if you would be able to meet with Bob Hall and Pat Clynes to discuss Producer Services,confirm to meet with Bob Hall and Pat Clynes,confirm to meet with Bob Hall and Pat Clynes
138,"Rajesh, fyi.",Please work with Paul and Dan on the document.,work with Paul and Dan on the document,work with Paul and Dan on document,work with Paul and Dan on document
315,"I thought you would enjoy this. PS bring the magic dollar to lunch! m. Gentlemen, I thought that all good Sooners would enjoy this.","If you know of others that I've left out, please forward it on.",forward this to others,forward it on,forward it on
368,"Holli:. Will you be on this call? Also, you never answered my question about reference market makers.",Please call me early today.,call SENDER,call sender,call sender
10,Attached is a draft of a memo summarizing the procedures we discussed at our meeting on Thursday. We will also be assisting in preparing the procedures. Please call or email if you have any questions or comments. Best regards. This email is sent by a law firm and contains information that may be privileged and confidential.,"If you are not the intended recipient, please delete the email and notify us immediately.",delete email and notify SENDER,delete email and notify sender,delete email and notify sender
257,I suspect a consent by the equipment seller may also be required. More to come on this one. As I learn more about the process I will keep you advised. Kay. I am attaching a revised draft notice of intent to exercise the purchase option for the equipment for the CA transactions. We need two separate notices - one for CA I and one for CA II.,"Ben/Fred/Rebecca, could you please prepare a draft of the two exhibits?",prepare draft of exhibits,prepare a draft of the two exhibits,prepare a draft of the two exhibits


In [97]:
df[['context', 'task', 'ref', 'decoded_1', 'Rouge-1_1', 'decoded_2', 'Rouge-1_2']].sample(10)

,context,task,ref,decoded_1,Rouge-1_1,decoded_2,Rouge-1_2
738,NaN,1. Presentations: can you please email your presentation to me by 15th,email presentation to SENDER,email presentation to sender,1.000000,email presentation to sender,1.000000
35,IMPORTANT:,PLEASE READ THE ATTACHED FILE Ethical Wall Procedure AND PRINT AND SIGN THE EMPLOYEE CERTIFICATION below .,"read file , print and sign certification",read attached file,0.400000,read attached file,0.400000
521,"John/John/Frank. As we are approaching the November 1st Trading Track interview dates, we need to finalize the reminder of the external candidate's initial screen.",Can you please provide me with your feedback/status.,provide feedback to SENDER,provide feedback to sender,1.000000,provide feedback,0.666667
721,"FYI, Attached is the form of lease agreement that Hanover has forwarded for our lease of the compression equipment from Hanover on the Oakhill deal. This is actually GE Capital's form of lease agreement, but would be executed between Hanover and Enron Compression Services Company.",Please review and provide any comments you may have.,review and comment,review and provide comments,0.571429,review and provide comments,0.571429
223,"Lisa, I was returning your phone call to Carol St. Clair about the agreement and deleted the message by mistake before I got to your phone no.",Can you give me a call at 713-8533-399 .,call SENDER at 713-8533-399,call sender at 713-8533-399,1.000000,call sender at 713-8533-399,1.000000
376,"Christie, I am forwarding you a message I have received from the University of Houston.",Can you help them?,help SENDER 's mesage,help them,0.333333,help them,0.333333
197,NaN,Please handle this.,handle this,handle it,0.500000,handle it,0.500000
500,"the information you have provided for disclosure schedules and will circulate a draft tonight or Friday morning for your further comments. Note that we are attempting to have the term Subsidiary defined to include only consolidated subsidiaires. We must be prepared, however, to include some representations about significant off-balance-sheet entities. Accordingly, as you review drafts of the agreement and the disclosure letter, please note any comments that apply only to unconsolidated entities.",Please return your comments to Rob Walls EB 5012 or John Ale 3AC 1023 by Friday at 3:00 pm.,send comments on drafts to Rob Walls or John Ale,return comments to Rob Walls EB or John Ale at 1023,0.666667,return comments to Rob Walls EB 5012 or John Ale,0.700000
68,guess it helps to attach the file. PL. Attached is a file we put together with the daily figures on them for Central 1 & Central 2. The total page should tie to what I gave you last week. I highlighted the five days on each month the highest absolute value change. I then attached those days P&L top sheets.,Please review this information and let us know what type of detailed support you will need.,review the P&L top sheets and respond to SENDER,review information,0.181818,review information,0.181818
255,"Duke shall subsequently subscribe to a five year transportation contract for ,000 Dth/d of East of Thoreau EOT capacity at a .05/Dth/d one-part rate effective November 1, 2000. Duke has been given alternate receipt and delivery rights in Transwestern's EOT Area. Duke has also been granted the ability to flow to California on an alternate basis at a total rate of .09/Dth/d. Please indicate approval via REPLY WITH HISTORY.",Please contact me at x5702 if any questions arise.,contact SENDER at x5702,contact sender at x5702,1.000000,contact sender at x5702,1.000000


In [98]:
same = (df['decoded_1'] == df['decoded_2'])
same.mean()

0.46

In [43]:
df[same].describe()

,refWC_1,decodedWC_1,taskWC_1,contextWC_1,Rouge-1_1,Rouge-2_1,Rouge-L_1,Rouge-1_1,Rouge-2_1,Rouge-L_1,refWC_2,decodedWC_2,taskWC_2,contextWC_2,Rouge-1_2,Rouge-2_2,Rouge-L_2,Rouge-1_2,Rouge-2_2,Rouge-L_2
count,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000
mean,4.748503,3.940120,11.269461,25.260479,0.612479,0.372191,0.610961,0.723598,0.558055,0.719970,4.748503,3.940120,11.269461,25.260479,0.612479,0.372191,0.610961,0.723598,0.558055,0.719970
std,2.627456,1.821462,7.115889,26.359967,0.269352,0.368519,0.271387,0.276246,0.387449,0.278054,2.627456,1.821462,7.115889,26.359967,0.269352,0.368519,0.271387,0.276246,0.387449,0.278054
min,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,2.000000,6.000000,0.000000,0.444444,0.000000,0.444444,0.500000,0.250000,0.500000,3.000000,2.000000,6.000000,0.000000,0.444444,0.000000,0.444444,0.500000,0.250000,0.500000
50%,4.000000,4.000000,9.000000,15.000000,0.600000,0.333333,0.600000,0.755952,0.500000,0.750000,4.000000,4.000000,9.000000,15.000000,0.600000,0.333333,0.600000,0.755952,0.500000,0.750000
75%,6.000000,5.000000,14.000000,49.750000,0.800000,0.666667,0.800000,1.000000,1.000000,1.000000,6.000000,5.000000,14.000000,49.750000,0.800000,0.666667,0.800000,1.000000,1.000000,1.000000
max,16.000000,14.000000,47.000000,75.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,14.000000,47.000000,75.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [107]:
df[~same][['context', 'task', 'ref', 'decoded_1', 'decoded_2']].sample(10)

,context,task,ref,decoded_1,decoded_2
112,This makes my cash balance at Reliant short by about 130.00 if prices post on the high side. My trading contact at Reliant said that his credit department DOES NOT want to work with us at all. My contact said his credit department could force him to cut the deal. We won't know until this afternoon or Monday.,So please try to get the wire out today.,get wire out today,get wire out today,get the wire out today
614,"Dear Vince, I am so grateful for your efforts. I really appreciate you taking time from your busy schedule. If you have not contacted Michael Maddox, it's even better. I have updated and re-worded my resume to better reflect my accomplishments.",Would you please contact Michael Maddox of CERA and forward my resume to him?,forward SENDER 's resume to micheal maddox,contact Michael Maddox of cera and forward my resume to him,contact Michael Maddox of cera and forward resume to him
486,NaN,Please look at the confirm and let me know if it is ok.,review the confirm,look at confirm and respond to sender,confirm that it is ok
432,"Imelda,",Please switch my sitara access from central to west and email me with my password.,switch SENDER 's sitara access,switch sender 's access from central to email,switch to [UNK]
349,NaN,Can you help with this one?,help SENDER,help with this,help sender
568,"YES I AM. What are the dates? And I love your outfit today. Are you coming to TEAL??????? ? According to my numbers, I show we have taken 67,560 out of storage through today. And we are taking out 12,995 2599 x 5 days from the 27th - 31st.",Could you double check my numbers please?,review numbers for SENDER,check numbers,double check sender 's numbers
8,"Can you please delete access to the financial trading agreement database to the following people:. Anita Fam Brenda Whitehead Elizabeth Sager Jason Peters Stephanie Sever Christian Yoder Lorraine Filomeno Shari Stack Susan Flynn Lisa Lees. Two people need to have their access changed from Legal Dept., to Credit Dept. therefore they can make Credit comments, not Legal comments : Tom Moran and Veronica Espinoza.","Also, please add a lawyer in our Group to the database with full edit access: Cheryl Nelson.",add Cheryl Nelson to the database,add a lawyer to sender,add a lawyer to database
581,"Cathy, can you send the following to everyone presenting at the offsite","Please assess your hiring needs for 2001, and include that information in your minute presentations.",go over hiring needs put info in presentation,assess hiring needs,assess hiring needs for 2001 needs
528,NaN,"98-1534 is on the 215K for 3/2000 but is only good for 3/1 - 3/1 in Sitara, on deal #163005.",confirm extension of deal,only good for 3/1,on deal #
705,"Your feedback plays an important role in the performance management process, and your participation is very critical to the success of Enron's Performance Management goals. Please provide feedback on the employees listed below by accessing the Performance Management System PEP and completing an online feedback form as described in the Performance Management Quick Reference Guide . You may begin your feedback input immediately. Please have all feedback forms completed by the date noted below.","If you have any questions regarding PEP or your responsibility in the process, please call the PEP Help Desk at the following numbers:",call the pep help desk,call pep help,call the pep help desk


In [104]:
df.describe()

,refWC_1,decodedWC_1,taskWC_1,contextWC_1,Rouge-1_1,Rouge-2_1,Rouge-L_1,refWC_2,decodedWC_2,taskWC_2,contextWC_2,Rouge-1_2,Rouge-2_2,Rouge-L_2
count,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,5.186667,4.796000,13.256000,26.169333,0.640206,0.444700,0.638767,5.186667,4.636000,13.256000,26.169333,0.643884,0.452471,0.640710
std,2.799930,2.250586,7.889664,26.640200,0.278676,0.362389,0.280888,2.799930,2.254095,7.889664,26.640200,0.281626,0.364565,0.285419
min,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,8.000000,0.000000,0.444444,0.027778,0.422932,3.000000,3.000000,8.000000,0.000000,0.444444,0.108359,0.428571
50%,5.000000,4.000000,12.000000,16.500000,0.666667,0.400000,0.666667,5.000000,4.000000,12.000000,16.500000,0.666667,0.400000,0.666667
75%,6.000000,6.000000,17.000000,52.000000,0.888889,0.727273,0.888889,6.000000,6.000000,17.000000,52.000000,0.888889,0.750000,0.909091
max,28.000000,14.000000,71.000000,75.000000,1.000000,1.000000,1.000000,28.000000,16.000000,71.000000,75.000000,1.000000,1.000000,1.000000


In [77]:
df[df['decodedWC_1'] > df['decodedWC_2']][['context', 'task', 'ref', 'decoded_1', 'decoded_2']] #.sample(10)

,context,task,ref,decoded_1,decoded_2
2,2. We will respond actively to your comments. We are going to do our best to cure some of the current problems that exist and put into action preventative processes that will help identify smaller problems before they become major ones. We will be asking people for help in resolving some of the issues that are collected in the survey - your active role will help us identify the best solutions.,So take a few minutes to check your pulse and tell us what you think and how you feel about your job and about Enron.,reflect on how you feel about Enron and job,check pulse and respond to sender about job and Enron,respond to sender about your job and about Enron
3,"if you don't think you need to be there, i can try to have you hooked in by phone. so let us know your thoughts. even if your position is still not to offer anything, this conference should provide a terrific opportunity to set the stage for the judge about our confidence in the merits of our pending motions. copies of the notices are on their way to all of you.",please get back to me at your earliest convenience with your thoughts on settlement and the conference.,respond to SENDER about settlement and conference,get back to sender,respond to sender
7,"okey dokey. Kay, please read this memo..Because you will be working with the originators and will be the first lawyer to know when the condition of the orders is filled you are probably the right person to blow the whistle for the 13D filing..Let me know if you think otherwise. Attached is the memo regarding the Schedule 13D filing requirements in connection with the FuelCell transaction.",Please call me at 713-2211-131 with your questions.,call SENDER at 713-2211-131,call sender at 713-2211-131 with questions,call sender at 713-2211-131
11,"As of this moment, 68 percent of divers taking the survey are not changing existing travel plans. Of the 32 percent who are changing their plans, about 90 percent are only changing the dates or the destination of their travel. That means only 10 percent of divers taking the survey are permanently canceling their trips.","Please do your best to support the airline, hospitality, and resort sectors of the world economy and book a dive trip soon.",book a dive trip,support airline to hospitality and resort sectors,support airline airline
15,"I have a customer that would like to be set up on Enron Online. He is a wimby and under our contract with him, he will back to back each trade with public Service Company of colorado. In other words, he doesn't have his own credit.","Could you set him up, at least with a temporary password?",set him up with password,"set him up , at least temporary password",set him up
...,...,...,...,...,...
730,The attached is a monthy table w/ a summary of line pressures and production.,Take a look and see if this is what you need.,examine the attached monthly table,examine summary of line,review summary
732,"Rhonda,",Can you provide me with the Agency Agreement or other documentation you may have received with respect to this agency relationship.,send documentation about this agency relationship to SENDER,provide sender with the Agency Agreement or other documentation,provide sender or other documentation
736,"Mary,","Can you please prepare a Unanimous Consent for Enron Midstream Services, L.L.C.",prepare unanimous consent for Enron midstream services,prepare unanimous consent for Enron midstream services,prepare unanimous consent
737,NaN,can you please ch the time to 2:00 p.m. not 2:00 a.m. for the exercise time.,change the time to 2:00 pm,ch time to [UNK],change exercise time


In [81]:
df[df['context'] == "Lisa J. Mellencamp Enron North America Corp. - Legal 1400 Smith St. Houston, TX 77002 Tel: 713-8537-986 Fax: 713-6463-393 email: Lisa.Mellencamp@enron.com. The attached presentation is intended as a resource for tomorrow's meeting concerning the restructuring of Enron's gas agreement with Sithe Independence."]

,context,task,ref,decoded_1,refWC_1,decodedWC_1,taskWC_1,contextWC_1,Rouge-1_1,Rouge-2_1,...,refWC_2,decodedWC_2,taskWC_2,contextWC_2,Rouge-1_2,Rouge-2_2,Rouge-L_2,Rouge-1_2,Rouge-2_2,Rouge-L_2
513,"Lisa J. Mellencamp Enron North America Corp. - Legal 1400 Smith St. Houston, TX 77002 Tel: 713-8537-986 Fax: 713-6463-393 email: Lisa.Mellencamp@enron.com. The attached presentation is intended as a resource for tomorrow's meeting concerning the restructuring of Enron's gas agreement with Sithe Independence.","If possible, please review before the meeting.",review before meeting,review meeting,3,2,7,42,0.8,0.0,...,3,2,7,42,0.8,0.0,0.8,0.8,0.0,0.8
